# Use the AzureML Profiler

In [ ]:
%pip install azure-mgmt-authorization

## 1. Connect to Azure Machine Learning Workspace

### 1.1 Import required libraries

In [53]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
    ComputeInstance,
    IdentityConfiguration, 
    Data,
    CommandJob
)
from azure.ai.ml import Input
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes
import random

### 1.2 Set workspace details

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

In [ ]:
subscription_id = "6fe1c377-b645-4e8e-b588-52e57cc856b2"
resource_group = "role-creation-sample"
workspace_name = "test-workspace"

### 1.3 Set variables

In [57]:
rand = random.randint(0, 100000)
endpoint_name=f"endpt-moe-{rand}" 
profiler_compute_name="profiler2"
profiler_compute_size="Standard_DS4_v2"

### 1.4 Get a handle to the workspace 

In [55]:
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

## 2. Make an endpoint to profile

### 2.1 Create an endpoint

In [58]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

### 2.2 Create a deployment

In [60]:
deployment = ManagedOnlineDeployment(
    name="openapi",
    endpoint_name=endpoint_name,
    model=Model(path="../model-1/model/sklearn_regression_model.pkl"),
    code_configuration=CodeConfiguration(
        code="openapi/code-custom", scoring_script="score.py"
    ),
    environment=Environment(
        image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference",
        conda_file="profiler/env.yml",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

ValidationException: No such file or directory: /home/azureuser/localfiles/azureml-examples-parent/profiler/azureml-examples/sdk/python/endpoints/online/managed/profiler/env.yml

In [ ]:
deployment = ml_client.online_deployments.begin_create_or_update(deployment).result()

## 3. Create a compute instance to run the profiler

### 3.1 Define the compute instance

In [ ]:
compute = ComputeInstance(
    name=profiler_compute_name,
    size="Standard_DS4_v2", 
    identity=IdentityConfiguration(type="system_assigned")
)

In [ ]:
compute = ml_client.compute.begin_create_or_update(compute).result()

### 3.3 Get Authorization Management Clients 

In [ ]:
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.v2018_01_01_preview.models import RoleDefinition
import uuid

role_definition_client = AuthorizationManagementClient(
    credential=credential,
    subscription_id=subscription_id,
    api_version="2018-01-01-preview",
)

from azure.mgmt.authorization.v2020_10_01_preview.models import (
    RoleAssignment,
    RoleAssignmentCreateParameters,
)

role_assignment_client = AuthorizationManagementClient(
    credential=credential,
    subscription_id=subscription_id,
    api_version="2020-10-01-preview",
)

### 3.4 Assign the Contributor role to the compute instance

In [ ]:
role_name = "Contributor"
scope = compute.id

role_defs = role_definition_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

role_assignment_client.role_assignments.create(
    scope=scope,
    role_assignment_name=str(uuid.uuid4()),
    parameters=RoleAssignmentCreateParameters(
        role_definition_id=role_def.id,
        principal_id=compute.identity.principal_id,
        principal_type="ServicePrincipal"
    )
)

## 4. Run the profiler

### 4.1 Upload the payload

In [47]:
payload = Data(
    name="payload",
    type=AssetTypes.URI_FILE,
    path="profiler/profiling/payload.txt",
    datastore="workspaceblobstore"
)

In [ ]:
payload = ml_client.data.create_or_update(payload)

### 4.2 Create a profiling job

In [ ]:
job = CommandJob(
    command="python -m online_endpoints_model_profiler --payload_path ${{inputs.payload}}", 
    experiment_name="profiling-job", 
    display_name=f"{profiler_compute_size}:1",
    environment=Environment(
        image="mcr.microsoft.com/azureml/online-endpoints-model-profiler:latest"
    ),
    environment_variables={ 
        "ONLINE_ENDPOINT": endpoint_name,
        "DEPLOYMENT": "blue",
        "PROFILING_TOOL": "wrk",
        "DURATION": "10",
        "CONNECTIONS": "1", 
        "TARGET_RPS": "50", 
        "CLIENTS": "1", 
        "TIMEOUT": "10",
        "THREAD": "1"
    },
    compute = f"azureml:{profiler_compute_name}",
    inputs = {
        "payload" : Input(
            type="uri_file", 
            path=payload.path 
        )
    }
)